In [1]:
#-*- coding:UTF-8 -*-
import pandas as pd
import networkx as nx
import numpy as np

import os
import datetime

#データのパス
data_path = "/Volumes/USB30/H30/JSC/Data/Podium Matches(2004-16)"

#競技種目
sport = "Judo"
discipline = "60kg"
gender = "Men"

#期間
start_date = "2010/1/1"
end_date = "2016/8/5"

start_date = datetime.datetime.strptime(start_date, "%Y/%m/%d")
end_date = datetime.datetime.strptime(end_date, "%Y/%m/%d")

In [25]:
#データ読み込み
Data = pd.DataFrame(None)

file = os.listdir(data_path)
file = [x for x in file if not x.startswith(".")] #隠しファイル対策

for i in file:
    d = pd.read_csv("{}/{}".format(data_path,i),engine="python")
    Data = pd.concat([Data,d.reset_index()])
    
Data = Data.reset_index(drop=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [26]:
Data

,Age,AgeDays,Competition,CompetitionSet,CompetitionSetDetail,Continent,Country,Discipline,DoB,Gender,...,Result,ResultOpp,ResultPart,Score,Sport,Team,Team Member,WDL,Year,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25-24 AET,24,25,"25-24 (8-8, 25-24) AET",Handball,Germany,No,W,2004,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24-25 AET,25,24,"24-25 (8-8, 24-25) AET",Handball,Ukraine,No,L,2004,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29-25,25,29,29-25 (14-13),Handball,Hungary,No,W,2004,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25-29,29,25,25-29 (13-14),Handball,Russia,No,L,2004,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25-27,27,25,25-27 (11-11),Handball,Denmark,No,L,2004,4
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27-25,25,27,27-25 (11-11),Handball,Norway,No,W,2004,5
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0-2,2,0,"9-15,9-15",Badminton,Blair / Munt,No,L,2004,6
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2-0,0,2,"15-9,15-9",Badminton,Zhang J / Gao L,No,W,2004,7
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0-2,2,0,"8-15,11-15",Badminton,Cai Y / Fu H F,No,L,2004,8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2-0,0,2,"15-8,15-11",Badminton,Hadiyanto / Alven,No,W,2004,9


In [3]:
#データ整形
df = Data.copy()

#Dateをdatetime型に変換
df["Date"] = pd.to_datetime(df["Date"])

df = df[df["Date"]>start_date]
df = df[df["Date"]<end_date]
df = df[df["Sport"]==sport]
df = df[df["Discipline"]==discipline]
df = df[df["Gender"]==gender]

df = df.sort_values("Date")
df = df.reset_index(drop=True)

In [7]:
import networkx as nx

def d_cen(d_graph):  
    pagerank=pd.DataFrame(pd.Series(nx.pagerank_numpy(d_graph,weight='weight',alpha=0.85),name='d_pagerank')).reset_index()
    ev=pd.DataFrame(pd.Series(nx.eigenvector_centrality(d_graph,weight='weight',max_iter=100000),name='d_ev')).reset_index()  
    d_cen_df=pd.concat([pagerank,ev['d_ev']],axis=1,ignore_index=True)
    d_cen_df.columns=['Person/Team','d_pagerank','d_ev']
    return d_cen_df

def  timeDec (count, t_start, t_end, beta):
    diff=(t_end-pd.to_datetime(t_start)).astype('timedelta64[D]')
    d_count= count*np.exp(-beta*(diff))
    return(d_count)

def d_elo(d_data,k0,beta):
    d_player_rating = pd.DataFrame(None)
    d_player_rating['Person/Team']=pd.concat([d_data["Participant"],d_data["Opponent"]]).drop_duplicates()
    #初期値
    d_player_rating["Rating"]=1500
    #diff=(pd.to_datetime(rank_data['Date'].index[len(rank_data)-1])-pd.to_datetime(rank_data['Date'])).astype('timedelta64[D]')
    diff=(max(d_data["Date"])-pd.to_datetime(d_data['Date'])).astype('timedelta64[D]')
    d_data["diff"] =diff

    for winner,loser,diff in zip(d_data["Participant"],d_data["Opponent"],d_data["diff"]):
        winner_rate = d_player_rating["Rating"][d_player_rating["Person/Team"]==winner].values[0]
        loser_rate = d_player_rating["Rating"][d_player_rating["Person/Team"]==loser].values[0]

        winner_exp_rate=1/(1+10**((loser_rate-winner_rate)/400))
        loser_exp_rate=1-winner_exp_rate

        k= k0*np.exp(-beta*(diff))

        d_player_rating['Rating'][d_player_rating['Person/Team']==winner]+=k*(1-winner_exp_rate)
        d_player_rating['Rating'][d_player_rating['Person/Team']==loser]+=k*(0-loser_exp_rate)        
    d_player_rating.columns = ["Person/Team","d_Elo"]    
    return d_player_rating

In [8]:
def Rating_to_Rank(data):
    person_team = data["Person/Team"]
    data = data.rank(ascending=False,method='min')[['Person/Team','d_pagerank', 'd_ev', 'd_Elo']]
    data["Person/Team"] = person_team
    return data

In [9]:
#ダイナミック
d_data_nx = df.groupby(["Opponent","Participant",'Date']).size().reset_index(name='count')
end_line = max(d_data_nx["Date"])
d_weight = timeDec(d_data_nx['count'],d_data_nx['Date'], end_line, 1/365)
d_data_nx['count']=d_weight
d_data_nx = d_data_nx.groupby(["Opponent","Participant"]).sum().reset_index()
d_data_tuples = [tuple(x) for x in d_data_nx.values]
d_graph = nx.DiGraph()
d_graph.add_weighted_edges_from(d_data_tuples,weight="weight")
dynamic_cen = d_cen(d_graph)
        
d_data_elo = df.groupby(["Opponent","Participant",'Date']).size().reset_index(name='count')
dynamic_elo = d_elo(d_data_elo,20,1/1440)
        
dynamic_rating = pd.merge(dynamic_cen,dynamic_elo)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
dynamic_rating.head()

,Person/Team,d_pagerank,d_ev,d_Elo
0,"A Lamusi (CHN, 02 Jun 1989)",0.000346,0.000127,1504.967632
1,"Amartuvshin Dashdavaa (MGL, 15 Dec 1987)",0.011656,0.202540,1841.942059
2,"Boldbaatar Chimed-Yondon (MGL, 20 Oct 1983)",0.000768,0.001842,1513.665310
3,"Choi Min-Ho (KOR, 18 Aug 1980)",0.000262,0.000002,1500.632431
4,"Eisa Majrashi (KSA, 19 Jul 1986)",0.005299,0.052926,1570.935985


In [ ]:
#世界ランキング
#期間内最新データ？
#欠損値どうする？


In [14]:
#ランキング化
rank_df = Rating_to_Rank(dynamic_rating)
rank_df["min"] = rank_df.mean(axis="columns")
rank_df.head()

,Person/Team,d_pagerank,d_ev,d_Elo,min
0,"A Lamusi (CHN, 02 Jun 1989)",494.0,502.0,98.0,364.666667
1,"Amartuvshin Dashdavaa (MGL, 15 Dec 1987)",5.0,5.0,1.0,3.666667
2,"Boldbaatar Chimed-Yondon (MGL, 20 Oct 1983)",264.0,297.0,68.0,209.666667
3,"Choi Min-Ho (KOR, 18 Aug 1980)",579.0,621.0,211.0,470.333333
4,"Eisa Majrashi (KSA, 19 Jul 1986)",58.0,59.0,29.0,48.666667
